# ReinforcementLearningBot training

In [3]:
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from time import perf_counter
from tqdm.notebook import trange
from copy import deepcopy
from itertools import combinations
import joblib

sys.path.append('..')
from env import BlockadeEnv
from players.ReinforcementLearningBot import ReinforcementLearningBot
from players.OptimizedBot import OptimizedBot
from players.HeuristicBot import HeuristicBot
from players.RandomBot import RandomBot

In [4]:
env = BlockadeEnv(arena_size=15, verbose=False)